# Transfer Learning

... following https://keras.io/guides/transfer_learning/

__Also interesting: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/__

In [ ]:
import os

import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model, save_model

In [ ]:
IMS = (150, 150)
BFS = 10
BTS = 32
CLASSES=['cat','dog']

### Load data(sub)sets

In [ ]:
tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(6)):
    ax = plt.subplot(3, 2, i + 1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis("off")

### Preprocess (augment) data

In [ ]:
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, IMS), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, IMS), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, IMS), y))

train_ds = train_ds.cache().batch(BTS).prefetch(buffer_size=BFS)
validation_ds = validation_ds.cache().batch(BTS).prefetch(buffer_size=BFS)
test_ds = test_ds.cache().batch(BTS).prefetch(buffer_size=BFS)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[i]))
        plt.axis("off")

## Define the model (as fixed feature extractor)

In [ ]:
SHAPE = (150, 150, 3)
base_model = keras.applications.Xception(
    weights="imagenet",
    input_shape=SHAPE,
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=SHAPE)
x = data_augmentation(inputs)

# Pre-trained Xception weights requires that input be normalized from (0, 255) to a range (-1., +1.), 
# the normalization layer does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode when we unfreeze the base 
# model for fine-tuning, so we make sure that the base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

### Train and save the model

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

In [ ]:
model.save('keras_TL_cat_dog') # tf.keras.model.save: Saves the model to Tensorflow SavedModel (default) or a single HDF5 file
loaded_model = load_model('keras_TL_cat_dog')
# model = load_model(filepath, compile = True)

## Fine-tune the model

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode since we passed `training=False` 
# when calling it. This means that the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.SGD(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

In [ ]:
save_model(model, 'fine_tuned_keras_TL_cat_dog-SGD') # tf.keras.models.save_model defaults to tensorflow savedmodel format in tf2

### Compare feature extraction and fine-tuned model

In [ ]:
loaded_model.evaluate(test_ds) # i.e. feature extractor [0.08429354429244995, 0.9720550179481506] 
model.evaluate(test_ds) # fine-tuned [0.07796552032232285, 0.9797936081886292]

*Apparently, fine-tuning the model improved its loss and accuracy by about 0.007*

## Predicting a batch of images using different activations

In [ ]:
def esc(code):
     return f'\033[{code}m'
    
funcs = {
    "relu": tf.nn.relu,
    "relu6": tf.nn.relu6,
    "selu": tf.nn.selu,
    "elu": tf.nn.elu,
    "swish": tf.nn.swish,
    "leaky-relu": tf.nn.leaky_relu,
    "sigmoid": tf.nn.sigmoid,
    "softsign": tf.nn.softsign,
    # not applicable: 
    # "crelu": tf.nn.crelu,
    # "softmax": tf.nn.softmax,
    # "log_softmax": tf.nn.log_softmax,
}

image_batch, label_batch = test_ds.as_numpy_iterator().next()
predictions = model.predict(image_batch).flatten()
predictions = tf.convert_to_tensor(predictions)
print(f'{label_batch}: Labels')

for func in funcs.keys():
    # Apply different activation functions to see which fits best
    function = funcs[func]
    predictions1 = function(predictions)
    predictions1 = tf.where(predictions1 < 0.5, 0, 1)
    if np.array_equal(predictions1.numpy(), label_batch): 
        # print predictions in reed if they comply with the labels
        print(esc('31;1;4') + predictions1.numpy() + esc(0) + ' : Predictions' + func)
    else:
        print(f'{predictions1.numpy()}: Predictions {func}')

*As we can see, all functions return the same predictions. The 5th prediction is always wrong.*

In [ ]:
plt.figure(figsize=(25, 25))
for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title(CLASSES[label_batch[i]])
    plt.axis("off")

### Predict a single image

In [ ]:
img = keras.preprocessing.image.load_img(
    "/Users/natalie/Downloads/iu-4.jpeg", target_size=IMS
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array, steps=1)
predictions = tf.nn.sigmoid(predictions)
predictions1 = tf.where(predictions < 0.5, 0, 1)
print(CLASSES[predictions1.numpy()[0][0]])

### Convert model to TFJS format

In [ ]:
!pip install tensorflowjs

In [ ]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, './tfjs_ft_cat_dog')

*Tensorflow provides an API which you can use to save your Python model to tfjs-format directly.* 

>For me, the output was not usable because it has some problems with functional and experimental layers.

*Alternatively, just save your model as shown above in SavedModel-format and use the command line tfjs converter thereafter.*

## Save model in GoogleDrive (Colab version only)

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
checkpoint_path = "/content/gdrive/My Drive/colab/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

*model.fit(train_ds, epochs=epochs, validation_data=validation_ds, callbacks=\[cp_callback\])*

*model.load_weights(checkpoint_path)*

*model.compile(
    optimizer=keras.optimizers.SGD(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)*

In [ ]:
model.save("/content/gdrive/My Drive/TL_SGD_cat-dog_colab" )

# Add prediction "decoder" directly into the model ?

In [ ]:
base_model = keras.applications.Xception(
    weights="imagenet",
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))

# Pre-trained Xception weights requires that input be normalized from (0, 255) to a range (-1., +1.), 
# the normalization layer does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(inputs)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode when we unfreeze the base 
# model for fine-tuning, so we make sure that the base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)